# **Method 2 - Using Sentence Transformer Model**

# **Install Library Package**

In [7]:
!pip install PyMuPDF transformers torch faiss-cpu
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 4.8 MB/s eta 0:00:00


# **Import Libraries**

In [8]:
import fitz  # PyMuPDF
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from typing import List, Tuple


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


# **Function to Extract Text from PDFs:**

In [9]:
def extract_text_from_pdfs(pdf_files: List[str]) -> List[str]:
    text_list = []
    for pdf_file in pdf_files:
        doc = fitz.open(pdf_file)
        for page in doc:
            text_list.append(page.get_text())
        doc.close()
    return text_list


# **Function to Create Embeddings**

In [10]:
def create_embeddings(text_list: List[str]) -> np.ndarray:
    model = SentenceTransformer('all-MiniLM-L6-v2')  # Use a model with high accuracy for text
    embeddings = model.encode(text_list, convert_to_numpy=True)
    return embeddings


# **Function to Create and Store a Vector Database:**

In [11]:
class VectorDatabase:
    def __init__(self):
        self.index = faiss.IndexFlatL2(384)  # Set to 384, which is the embedding size for all-MiniLM-L6-v2
        self.texts = []

    def add(self, embeddings: np.ndarray, texts: List[str]):
        self.index.add(embeddings)
        self.texts.extend(texts)

    def search(self, query_vector: np.ndarray, k: int = 5) -> List[Tuple[str, float]]:
        distances, indices = self.index.search(query_vector, k)

        # Ensure that indices are accessed correctly
        results = [(self.texts[indices[0][i]], distances[0][i]) for i in range(len(indices[0]))]
        return results



# **Function to Answer Queries:**

In [13]:
def answer_query(query: str, vector_db: VectorDatabase) -> str:
    model = SentenceTransformer('all-MiniLM-L6-v2')
    query_vector = model.encode([query])
    results = vector_db.search(query_vector)

    # Combine the retrieved passages into a coherent answer
    response = "\n\n".join([result[0] for result in results])
    return response

# **Main Flow Implementation**

In [14]:
# List of PDF file paths to process
pdf_paths = ['/content/drive/MyDrive/engAIge GmbH/BMF_2013_07_24.pdf', '/content/drive/MyDrive/engAIge GmbH/BMF_2017_12_06.pdf', '/content/drive/MyDrive/engAIge GmbH/BMF_2017_12_21.pdf', '/content/drive/MyDrive/engAIge GmbH/BMF_2021_08_12.pdf', '/content/drive/MyDrive/engAIge GmbH/BMF_2023_10_05.pdf']  # Replace with your PDF paths

# Step 1: Extract text
extracted_texts = extract_text_from_pdfs(pdf_paths)

# Step 2: Create embeddings
embeddings = create_embeddings(extracted_texts)

# Step 3: Create a vector database and add the embeddings
vector_db = VectorDatabase()
vector_db.add(embeddings, extracted_texts)

# List of queries to answer
queries = [
    "Wie hoch ist die Grundzulage?",
    "Wie werden Versorgungsleistungen aus einer Direktzusage oder einer Unterstützungskasse steuerlich behandelt?",
    "Wie werden Leistungen aus einer Direktversicherung, Pensionskasse oder einem Pensionsfonds in der Auszahlungsphase besteuert?"
]

# Get answers for each query
for query in queries:
    answer = answer_query(query, vector_db)
    print(f"Answer to the query '{query}':")
    print(answer)
    print("\n" + "="*80 + "\n")

Answer to the query 'Wie hoch ist die Grundzulage?':
 
Seite 19  
 
46 
Jeder unmittelbar oder mittelbar Zulageberechtigte erhält auf Antrag für seine im abge-
laufenen Beitragsjahr gezahlten Altersvorsorgebeiträge eine Grundzulage. Für die Zulagen-
gewährung bei mittelbar zulageberechtigten Ehegatten/Lebenspartnern sind die Rzn. 26 bis 29 
zu beachten. Die Grundzulage beträgt ab dem Beitragsjahr 2018 jährlich 175 €. 
 
47 
Für unmittelbar Zulageberechtigte, die das 25. Lebensjahr noch nicht vollendet haben, erhöht 
sich die Grundzulage einmalig um einen Betrag von 200 € (sog. Berufseinsteiger-Bonus). Für 
die Erhöhung ist kein gesonderter Antrag erforderlich. Die erhöhte Grundzulage ist einmalig 
für das erste nach dem 31. Dezember 2007 beginnende Beitragsjahr zu zahlen, für das der 
Zulageberechtigte die Altersvorsorgezulage beantragt, wenn er zu Beginn des betreffenden 
Beitragsjahres das 25. Lebensjahr noch nicht vollendet hat. Das Datum des Vertragsab-
schlusses ist insoweit unerh